In [1]:
import requests
from env import env
import urllib.parse
from datetime import datetime

In [2]:
csUrl = "https://www.cheapshark.com/api/1.0/"

def supCheapSharkExtractor(path,**kwargs) -> list:

    send = csUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"
    return requests.get(send).json()

def getGameByTheme(title:str = None,size:int = 60) -> list:
    """
    Retorna uma lista de jogos baseados em um tema.
    """
    if title: title = urllib.parse.quote(title)
    response = supCheapSharkExtractor(path="games?",title=title,size=size)
    response_id = []

    for i in response: 
        if i['steamAppID'] != None: 
            response_id.append(f"{i['external']} - {i['steamAppID']}")

    return response_id

In [3]:
stUrl = "https://store.steampowered.com/api/"

def supSteamPoweredExtractor(path,**kwargs) -> dict:
    send = stUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"

    return requests.get(send).json()

def getGameOverview(appids:str = None):
    response = supSteamPoweredExtractor(path="appdetails?",appids=appids,cc="pt-br",l="portuguese")
    print(response)
    image_v = response[appids]['data']

    desciption_v = response[appids]["data"]['detailed_description']

    dev_v = "Desenvolvido pela: " + response[appids]["data"]['developers'][0]
    pub_v = "Distribuído pela: " + response[appids]["data"]['publishers'][0]
    
    try:
        price_v = "Preço na steam: "+ response[appids]["data"]['price_overview']['final_formatted']
    except:
        price_v = "Preço indisponível"

    genre_v = response[appids]['data']['genres'][0]['description'] # more tests

    date_v = response[appids]['data']['release_date']['date']

    try:
        meta_v = "Metacritic: " + str(response[appids]["data"]['metacritic']['score'])
    except Exception as e:
        meta_v = "Metacritic não disponível"

    try:
        recomendations_v = "Número de avaliações na steam: "+str(response[appids]["data"]["recommendations"]['total'])
    except:
        recomendations_v = "Produto não avaliado"

    min_rec_v = "Requisitos mínimos: " + response[appids]['data']['pc_requirements']['minimum']

    max_rec_v = "Requisitos recomendados: " + response[appids]['data']['pc_requirements']['recommended']


    struct = {
        "desc": desciption_v,
        "dev": dev_v,
        "pub": pub_v,
        "price":price_v,
        "genre":genre_v,
        "date":date_v,
        "meta":meta_v,
        "recomendations":recomendations_v,
        "min_rec":min_rec_v,
        "max_rec":max_rec_v
    }

    return struct

In [26]:
adUrl = "https://api.isthereanydeal.com/"

def supIsThereAnyDealExtractor(path,**kwargs) -> list:

    send = adUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"

    return requests.get(send).json()

def getGameLowest(gameid:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/overview/?",key=env("AnyDealKey"),country="BR",shop="steam",ids=f"app/{gameid}")

    print(type(response['data'][f"app/{gameid}"]))

    try:

        store_today = response['data'][f"app/{gameid}"]['price']['store']
        price_today = response['data'][f"app/{gameid}"]['price']['price_formatted']

    except:
        store_today = None
        price_today = None

    store_hist = response['data'][f"app/{gameid}"]['lowest']['store']
    price_hist = response['data'][f"app/{gameid}"]['lowest']['price_formatted']

    price_url = response['data'][f"app/{gameid}"]['urls']['history']

    data_formatada = datetime.utcfromtimestamp(response['data'][f"app/{gameid}"]['lowest']['recorded']).strftime('%d-%m-%Y') # dia do menor preço histórico

    struct = {
        "store_today":store_today,
        "price_today":price_today,
        "store_hist":store_hist,
        "price_hist":price_hist,
        "price_url":price_url,
        "date":data_formatada
    }

    return struct

def supGetGamePlains(gameid:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/plain/id/?",key=env("AnyDealKey"),shop="steam",ids=f"app/{gameid}")
    return response['data'][f"app/{gameid}"]

def supGetGamePrices(gamePlain:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/prices/?",key=env("AnyDealKey"),plains=gamePlain,country="BR")

    price_list = []
    #try:
    for i in response['data'][gamePlain]['list']:
        price_list.append(f"{i['shop']['name']}: {i['price_old']} -> {i['price_new']} ({i['price_cut']}%)")
    #except Exception as e:
    #    return []

    return price_list

def getGamePrices(gameid: str = None):
    gamePlain = supGetGamePlains(gameid)
    return supGetGamePrices(gamePlain)


## Escolha do tema e jogo

In [5]:
response = getGameByTheme(title="outer")
for i in response:
    print(i)

Outerverse - 1293540
Outer Wilds - 753640
Outer Terror - 2189860
Outerstellar - 1170940
Outer Frontier - 1393850
DRAG Outer Zones - 773840
The Outer Worlds - 578650
Outer Space Shack - 1620870
Apollo in Outer Space - 1622840
Albedo: Eyes from Outer Space - 277520
Rescue Team: Danger from Outer Space! - 1333190
The Outer Worlds: Spacers Choice Edition - 1920490
Plan B From Outer Space: A Bavarian Odyssey - 1683620
Dexter Stardust : Adventures in Outer Space - 1647390
Captain ToonHead vs the Punks from Outer Space - 942390


## Retorno do overview

In [6]:
gameid = "753640"
game_overview = getGameOverview(appids=gameid)

{'753640': {'success': True, 'data': {'type': 'game', 'name': 'Outer Wilds', 'steam_appid': 753640, 'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': [1286760, 1622100], 'detailed_description': '<h1>You Might Also Like</h1><p><a href="https://store.steampowered.com/app/1497440/COCOON/" target="_blank" rel=""  id="dynamiclink_0" >https://store.steampowered.com/app/1497440/COCOON/</a><br><a href="https://store.steampowered.com/app/2008920/Lorelei_and_the_Laser_Eyes/" target="_blank" rel=""  id="dynamiclink_1" >https://store.steampowered.com/app/2008920/Lorelei_and_the_Laser_Eyes/</a></p><br><h1>Acerca do Jogo</h1>Winner of Best Game at the 2020 BAFTA Games Awards and named Game of the Year 2019 by Giant Bomb, Polygon, Eurogamer, and The Guardian, <i>Outer Wilds</i> is a critically-acclaimed and award-winning open world mystery about a solar system trapped in an endless time loop.<br><br><strong> Welcome to the Space Program!</strong><br>You\'re the newest recruit 

In [7]:
print(game_overview)

{'desc': '<h1>You Might Also Like</h1><p><a href="https://store.steampowered.com/app/1497440/COCOON/" target="_blank" rel=""  id="dynamiclink_0" >https://store.steampowered.com/app/1497440/COCOON/</a><br><a href="https://store.steampowered.com/app/2008920/Lorelei_and_the_Laser_Eyes/" target="_blank" rel=""  id="dynamiclink_1" >https://store.steampowered.com/app/2008920/Lorelei_and_the_Laser_Eyes/</a></p><br><h1>Acerca do Jogo</h1>Winner of Best Game at the 2020 BAFTA Games Awards and named Game of the Year 2019 by Giant Bomb, Polygon, Eurogamer, and The Guardian, <i>Outer Wilds</i> is a critically-acclaimed and award-winning open world mystery about a solar system trapped in an endless time loop.<br><br><strong> Welcome to the Space Program!</strong><br>You\'re the newest recruit of Outer Wilds Ventures, a fledgling space program searching for answers in a strange, constantly evolving solar system.<br><br><img src="https://cdn.akamai.steamstatic.com/steam/apps/753640/extras/OW1.png?t=1

In [ ]:
#print(game_overview[gameid]['success'])

#print(game_overview[gameid]["data"]['is_free'])

print(game_overview[gameid]["data"]['detailed_description']) #

print("Desenvolvido pela: " + game_overview[gameid]["data"]['developers'][0]) # 
print("Distribuído pela: " + game_overview[gameid]["data"]['publishers'][0]) #

print("Preço na steam: "+game_overview[gameid]["data"]['price_overview']['final_formatted']) #


print(game_overview[gameid]['data']['genres'][0]['description']) #

print(game_overview[gameid]['data']['release_date']['date']) #

try:

    print("Metacritic: " + str(game_overview[gameid]["data"]['metacritic']['score'])) #

except Exception as e:
    print("Metacritic não disponível") #

try:
    print("Número de avaliações na steam: "+str(game_overview[gameid]["data"]["recommendations"]['total'])) #
except:
    print("Produto não avaliado") #

print("Requisitos mínimos: " + game_overview[gameid]['data']['pc_requirements']['minimum'])

print("Requisitos recomendados: " + game_overview[gameid]['data']['pc_requirements']['recommended'])

#game_overview[gameid]['data']

## Retorno menor preço atual e histórico

In [8]:
game_lowest = getGameLowest(gameid)
game_lowest

<class 'dict'>


{'store_today': 'Epic Game Store',
 'price_today': 'R$59.99',
 'store_hist': 'Nuuvem',
 'price_hist': 'R$24.99',
 'price_url': 'https://isthereanydeal.com/game/outerwilds/history/',
 'date': '14-04-2022'}

In [ ]:
print(type(game_lowest['data'][f"app/{gameid}"]))

try:

    print(game_lowest['data'][f"app/{gameid}"]['price']['store']) # today´s lowest
    print(game_lowest['data'][f"app/{gameid}"]['price']['price_formatted'])

except:
    print("today == historical")

print(game_lowest['data'][f"app/{gameid}"]['lowest']['store']) # historical lowest
print(game_lowest['data'][f"app/{gameid}"]['lowest']['price_formatted'])


print(game_lowest['data'][f"app/{gameid}"]['urls']['history']) # price history site

print(game_lowest['data'][f"app/{gameid}"]['lowest']['recorded'])

data_formatada = datetime.utcfromtimestamp(game_lowest['data'][f"app/{gameid}"]['lowest']['recorded']).strftime('%d-%m-%Y') # dia do menor preço histórico

print(data_formatada)

## Retorno todos os preços do jogo

In [27]:
#plainResponse = getGamePlains(gameid=gameid)
#print(plainResponse)
#game_plain = plainResponse['data'][f"app/{gameid}"]

#print(game_plain)

pricesResponse = getGamePrices(gameid)

In [29]:
game_lowest = getGameLowest(gameid)
game_lowest

<class 'dict'>


{'store_today': 'Epic Game Store',
 'price_today': 'R$59.99',
 'store_hist': 'Nuuvem',
 'price_hist': 'R$24.99',
 'price_url': 'https://isthereanydeal.com/game/outerwilds/history/',
 'date': '14-04-2022'}

In [28]:
pricesResponse

#pricesResponse['data'][plainResponse]

['Epic Game Store: 59.99 -> 59.99 (0%)',
 'Gamesload: 59.99 -> 59.99 (0%)',
 'Steam: 59.99 -> 59.99 (0%)',
 'GamersGate: 59.99 -> 59.99 (0%)',
 'Nuuvem: 59.99 -> 59.99 (0%)',
 'Dreamgame: 59.99 -> 59.99 (0%)',
 'GreenManGaming: 59.99 -> 59.99 (0%)',
 'GameBillet: 122.369 -> 83.1949 (32%)',
 'Microsoft Store: 92.45 -> 92.45 (0%)',
 'Fanatical: 122.516 -> 93.885 (23%)',
 'Humble Store: 122.444 -> 122.444 (0%)',
 'WinGameStore: 122.601 -> 122.601 (0%)',
 'IndieGala Store: 123.403 -> 123.403 (0%)']

In [ ]:
#pricesResponse['data'][game_plain]['list']
print(pricesResponse)

price_list = pricesResponse['data'][game_plain]['list']
try:
    if price_list == []:
        print("sem lista")
    else:
        for i in pricesResponse['data'][game_plain]['list']:
            print(i['price_old'],i['price_new'],str(i['price_cut'])+"%",i['shop']['name'])
except Exception as e:
    print("sem lista")